In [1]:
import requests, json
import pandas as pd

In [2]:
def list_of_dicts_to_nested_dict(data):
    nested_dict = {}
    for item in data:
        current_dict = nested_dict
        for key, value in item.items():
            if key == 'agregados':
                if key not in current_dict:
                    current_dict[key] = []
                current_dict[key].extend(value)
            else:
                if key not in current_dict:
                    current_dict[key] = {}  # Create an empty nested dictionary if key doesn't exist
                current_dict = current_dict[key]
        if 'agregados' in current_dict:
            current_dict['agregados'] = list_of_dicts_to_nested_dict(current_dict['agregados'])
    return nested_dict

In [3]:
# Código do municipio Embu-Guaçu na base IBGE
cod_ibge = '3515103'

# Dados demográficos - IBGE

## População de Embu-Guaçu (2010-2022)

In [4]:
url_base = 'https://servicodados.ibge.gov.br/api/v3/agregados'

## Listar dados disponíveis para município

In [5]:
# identificador nivel municipal
nivel = 'N6'

response = requests.get(f'{url_base}?nivel={nivel}')

data_json = response.json()

for conjunto_dataset in data_json:
    print(conjunto_dataset['nome'], conjunto_dataset['id'])

Áreas Urbanizadas D5
Cadastro Central de Empresas CL
Censo Agropecuário CA
Censo Demográfico CD
Contagem da População  CM
Estatísticas dos Cadastros de Microempreendedores Individuais D9
Estimativas de População XF
Fundações Privadas e Associações Sem Fins Lucrativos FP
Indicadores de Desenvolvimento Sustentável IU
Índice de Preços ao Consumidor em Real IL
Índice de Reajuste do Salário Mínimo IJ
Índice Nacional de Preços ao Consumidor PC
Índice Nacional de Preços ao Consumidor Amplo IA
Índice Nacional de Preços ao Consumidor Amplo 15 IQ
Índice Nacional de Preços ao Consumidor Amplo Especial IG
Índice Nacional de Preços ao Consumidor Especial PG
Objetivos de Desenvolvimento Sustentável C2
Pesquisa da Pecuária Municipal PP
Pesquisa de Estoques ES
Pesquisa de Informações Básicas Municipais PM
Pesquisa de Orçamentos Familiares OF
Pesquisa de Serviços de Hospedagem SH
Pesquisa Estatísticas do Registro Civil RC
Pesquisa Nacional de Saneamento Básico SB
Pesquisa Nacional de Saúde XN
Pesquisa 

## Filtro de dasets relevantes

In [6]:
datasets_selecionados = ['CD','CM','XF','PM','OF','SB','XN','AA']

In [7]:
conj_datasets_selecionados = [dataset for dataset in data_json if dataset['id'] in datasets_selecionados]

In [8]:
conj_datasets_selecionados[0]

{'id': 'CD',
 'nome': 'Censo Demográfico',
 'agregados': [{'id': '102',
   'nome': 'Mulheres de 10 anos ou mais de idade, que viviam em união conjugal, total e que tiveram filhos, e os filhos tidos pelas mulheres de 10 anos ou mais de idade, que viviam em união conjugal, os nascidos vivos, os nascidos vivos no período de referência de 12 meses, nascidos mortos e os que estavam vivos na data de referência, segundo a situação do domicílio, a natureza da união conjugal e os grupos de idade das mulheres'},
  {'id': '103',
   'nome': 'Mulheres de 10 anos ou mais de idade, que viviam em união conjugal e tiveram filhos nascidos vivos, por número de filhos tidos nascidos vivos, segundo a situação do domicílio, a natureza da união conjugal e os grupos de idade das mulheres'},
  {'id': '105',
   'nome': 'Filhos tidos nascidos vivos das mulheres de 10 anos ou mais de idade, que viviam em união conjugal, segundo a situação do domicílio, a natureza da união conjugal e os grupos de idade das mulhere

## Listar anos disponíveis para cada conjunto de datasets

In [9]:
dict_anos_datasets = {}

for dataset in datasets_selecionados:
    list_anos = []

    response = requests.get(f'{url_base}?acervo=/S/{dataset}/P/Q')
    data_json = response.json()

    for ano in data_json:
        list_anos.append(ano['id'])

    dict_anos_datasets[dataset] = list_anos

In [10]:
dict_anos_datasets

{'CD': ['1872',
  '1890',
  '1900',
  '1920',
  '1940',
  '1950',
  '1960',
  '1970',
  '1980',
  '1991',
  '2000',
  '2010',
  '2022'],
 'CM': ['1996', '2007'],
 'XF': ['2001',
  '2002',
  '2003',
  '2004',
  '2005',
  '2006',
  '2008',
  '2009',
  '2011',
  '2012',
  '2013',
  '2014',
  '2015',
  '2016',
  '2017',
  '2018',
  '2019',
  '2020',
  '2021'],
 'PM': ['2005', '2020', '2021'],
 'OF': ['1987', '1995', '2002', '2008', '2018'],
 'SB': ['2000', '2008', '2017'],
 'XN': ['2013', '2019'],
 'AA': ['2009', '2012', '2015']}

In [11]:
# Neste caso, vamos trabalhar apenas com datasets que tiveram atualização recente
anos_datasets_selecionados = {'CD':'2022','XF':'2021','PM':'2021','XN':'2019'}

## Listar tabelas disponíveis para os anos e datasets selecionados

In [12]:
dicionario_tabelas_datasets = {}

for dataset in anos_datasets_selecionados.items():
    if isinstance(dataset[1], list):
        print(f'{url_base}?acervo=/S/{dataset[0]}/P/{dataset[1][0]}/P/{dataset[1][1]}/T/Q?_1700211710044')
        response = requests.get(f'{url_base}?acervo=/S/{dataset[0]}/P/1_{dataset[1][0]}/P/1_{dataset[1][1]}/T/Q?_=1700211710044')
        data_json = response.json()
        
        dicionario_tabelas_datasets[dataset[0]] = data_json
    else:
        periodo = f'1_{dataset[1]}'
        print(f'{url_base}?acervo=/S/{dataset[0]}/P/1_{dataset[1]}/T/Q?_=1700211710044')
        response = requests.get(f'{url_base}?acervo=/S/{dataset[0]}/P/1_{dataset[1]}/T/Q?_=1700211710044')
        data_json = response.json()
        dicionario_tabelas_datasets[dataset[0]] = data_json

https://servicodados.ibge.gov.br/api/v3/agregados?acervo=/S/CD/P/1_2022/T/Q?_=1700211710044
https://servicodados.ibge.gov.br/api/v3/agregados?acervo=/S/XF/P/1_2021/T/Q?_=1700211710044
https://servicodados.ibge.gov.br/api/v3/agregados?acervo=/S/PM/P/1_2021/T/Q?_=1700211710044
https://servicodados.ibge.gov.br/api/v3/agregados?acervo=/S/XN/P/1_2019/T/Q?_=1700211710044


Após exploração dos datasets no site do [IBGE](https://servicodados.ibge.gov.br/api/docs/agregados?versao=3), foi constatado que apenas o dataset do Censo Demográfico (CD) realmente possui dados atuais e representativos do contexto específico de Embu-Guaçu

# Requisições dados - Censo Demográfico IBGE

In [13]:
# Dicionário com tabelas disponíveis para requisição
dicionario_tabelas_datasets['CD'][:4]

[{'id': '1209', 'literal': 'População, por grupos de idade'},
 {'id': '4709',
  'literal': 'População residente, Variação absoluta de população residente e Taxa de crescimento geométrico'},
 {'id': '4711', 'literal': 'Domicílios recenseados, por espécie'},
 {'id': '4712',
  'literal': 'Domicílios particulares permanentes ocupados, Moradores em domicílios particulares permanentes ocupados e Média de moradores em domicílios particulares permanentes ocupados'}]

In [14]:
df_tabelas_censo = pd.json_normalize(dicionario_tabelas_datasets['CD'])

Lista de tabelas consultadas

- 9514- População residente, por sexo, idade e forma de declaração da idade
- 9520- População residente, Variação absoluta da população residente e Taxa de crescimento geométrico (segunda apuração)
- 9606- População residente, por cor ou raça, segundo o sexo e a idade
- 9521- Domicílios particulares permanentes ocupados, Moradores em domicílios particulares permanentes ocupados e Média de moradores em domicílios particulares permanentes ocupados (segunda apuração)
- 9397- Moradores em domicílios particulares permanentes ocupados, por existência de banheiro ou sanitário e número de banheiros de uso exclusivo do domicílio e por tipo de esgotamento sanitário, segundo grupos de idade e cor ou raça
- 9522- População residente, Área territorial e Densidade demográfica (segunda apuração)

In [15]:
# Lista de códigos das tabelas que serão consultadas na API
list_tabelas_censo = [str(x) for x in [9514, 9520, 9606, 9521, 9397, 9522]]
list_tabelas_censo

['9514', '9520', '9606', '9521', '9397', '9522']

## Lista de metadados para cada tabela

In [16]:
url_base

'https://servicodados.ibge.gov.br/api/v3/agregados'

In [17]:
dicionario_variaveis_tabelas = {}

for tabela in list_tabelas_censo:
    response = requests.get(f'{url_base}/{tabela}/metadados')
    data_json = response.json()['variaveis']

    print(data_json)

    dicionario_variaveis_tabelas[tabela] = {}

    for item in data_json:
        dicionario_variaveis_tabelas[tabela][str(item['id'])] = item['nome']

[{'id': 93, 'nome': 'População residente', 'unidade': 'Pessoas', 'sumarizacao': []}, {'id': 1000093, 'nome': 'População residente - percentual do total geral', 'unidade': '%', 'sumarizacao': []}]
[{'id': 93, 'nome': 'População residente', 'unidade': 'Pessoas', 'sumarizacao': []}, {'id': 5936, 'nome': 'Variação absoluta da população residente 2010 compatibilizada', 'unidade': 'Pessoas', 'sumarizacao': []}, {'id': 10605, 'nome': 'Taxa de crescimento geométrico', 'unidade': '%', 'sumarizacao': []}]
[{'id': 93, 'nome': 'População residente', 'unidade': 'Pessoas', 'sumarizacao': ['nivelTerritorial']}, {'id': 1000093, 'nome': 'População residente - percentual do total geral', 'unidade': '%', 'sumarizacao': []}]
[{'id': 381, 'nome': 'Domicílios particulares permanentes ocupados', 'unidade': 'Domicílios', 'sumarizacao': []}, {'id': 382, 'nome': 'Moradores em domicílios particulares permanentes ocupados', 'unidade': 'Pessoas', 'sumarizacao': []}, {'id': 5930, 'nome': 'Média de moradores em domi

In [18]:
dicionario_variaveis_tabelas

{'9514': {'93': 'População residente',
  '1000093': 'População residente - percentual do total geral'},
 '9520': {'93': 'População residente',
  '5936': 'Variação absoluta da população residente 2010 compatibilizada',
  '10605': 'Taxa de crescimento geométrico'},
 '9606': {'93': 'População residente',
  '1000093': 'População residente - percentual do total geral'},
 '9521': {'381': 'Domicílios particulares permanentes ocupados',
  '382': 'Moradores em domicílios particulares permanentes ocupados',
  '5930': 'Média de moradores em domicílios particulares permanentes ocupados'},
 '9397': {'382': 'Moradores em domicílios particulares permanentes ocupados',
  '1000382': 'Moradores em domicílios particulares permanentes ocupados - percentual do total geral'},
 '9522': {'93': 'População residente',
  '6318': 'Área da unidade territorial',
  '614': 'Densidade demográfica'}}

In [19]:
pd.json_normalize(dicionario_variaveis_tabelas)

,9514.93,9514.1000093,9520.93,9520.5936,9520.10605,9606.93,9606.1000093,9521.381,9521.382,9521.5930,9397.382,9397.1000382,9522.93,9522.6318,9522.614
0,População residente,População residente - percentual do total geral,População residente,Variação absoluta da população residente 2010 ...,Taxa de crescimento geométrico,População residente,População residente - percentual do total geral,Domicílios particulares permanentes ocupados,Moradores em domicílios particulares permanent...,Média de moradores em domicílios particulares ...,Moradores em domicílios particulares permanent...,Moradores em domicílios particulares permanent...,População residente,Área da unidade territorial,Densidade demográfica


# 9514: População residente, por sexo, idade e forma de declaração da idade

In [20]:
tabela = list_tabelas_censo[0]
variaveis = list(dicionario_variaveis_tabelas[tabela].keys())
str_variaveis = '|'.join(variaveis)

print(tabela)
print(str_variaveis)

9514
93|1000093


In [21]:
url_final = f'{url_base}/{tabela}/periodos/2022/variaveis/{str_variaveis}?localidades=N6[{cod_ibge}]&classificacao=2[all]|287[all]'
print(url_final)
response = requests.get(url_final)
data_json = response.json()

print(data_json)

https://servicodados.ibge.gov.br/api/v3/agregados/9514/periodos/2022/variaveis/93|1000093?localidades=N6[3515103]&classificacao=2[all]|287[all]


[{'id': '93', 'variavel': 'População residente', 'unidade': 'Pessoas', 'resultados': [{'classificacoes': [{'id': '2', 'nome': 'Sexo', 'categoria': {'6794': 'Total'}}, {'id': '287', 'nome': 'Idade', 'categoria': {'100362': 'Total'}}, {'id': '286', 'nome': 'Forma de declaração da idade', 'categoria': {'113635': 'Total'}}], 'series': [{'localidade': {'id': '3515103', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Embu-Guaçu (SP)'}, 'serie': {'2022': '66970'}}]}, {'classificacoes': [{'id': '2', 'nome': 'Sexo', 'categoria': {'6794': 'Total'}}, {'id': '287', 'nome': 'Idade', 'categoria': {'93070': '0 a 4 anos'}}, {'id': '286', 'nome': 'Forma de declaração da idade', 'categoria': {'113635': 'Total'}}], 'series': [{'localidade': {'id': '3515103', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Embu-Guaçu (SP)'}, 'serie': {'2022': '4079'}}]}, {'classificacoes': [{'id': '2', 'nome': 'Sexo', 'categoria': {'6794': 'Total'}}, {'id': '287', 'nome': 'Idade', 'categoria': {'6557': 'Menos de

In [22]:
municipio = 'Embu-Guaçu'
uf = 'SP'

list_results = []

for result in data_json[0]['resultados']:
    dict_result = {'municipio':municipio,'uf':uf}

    for classif in result['classificacoes']:
        var_name = classif['nome']
        var_class = list(classif['categoria'].values())[0]
        var_value = float(list(result['series'][-1]['serie'].values())[0])
        
        if (' a ' in var_class) or (var_class in ['Total','Homens','Mulheres']):
            dict_result[var_name] = var_class
        else:
            pass

    if 'Idade' in list(dict_result.keys()):
        dict_result['habitantes'] = var_value
        list_results.append(dict_result)

df_censo = pd.DataFrame(list_results)
df_censo.columns = ['municipio','uf','sexo','idade','declaracao_idade','habitantes']
df_censo.drop(columns=['declaracao_idade'], inplace=True)
df_censo = df_censo.query('idade != "Total" & sexo != "Total"')

df_censo.to_csv('./data/processed/populacao_embu_guacu_2022_sexo_idade.csv', index=False)

In [23]:
df_censo['habitantes'].sum()

66964.0

# 9520- População residente, Variação absoluta da população residente e Taxa de crescimento geométrico (segunda apuração)

In [24]:
tabela = list_tabelas_censo[1]
variaveis = list(dicionario_variaveis_tabelas[tabela].keys())
str_variaveis = '|'.join(variaveis)

print(tabela)
print(str_variaveis)

9520
93|5936|10605


In [25]:
url_final = f'{url_base}/{tabela}/periodos/2022/variaveis/{str_variaveis}?localidades=N6[{cod_ibge}]'
print(url_final)
response = requests.get(url_final)
data_json = response.json()

print(data_json)

https://servicodados.ibge.gov.br/api/v3/agregados/9520/periodos/2022/variaveis/93|5936|10605?localidades=N6[3515103]
[{'id': '93', 'variavel': 'População residente', 'unidade': 'Pessoas', 'resultados': [{'classificacoes': [], 'series': [{'localidade': {'id': '3515103', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Embu-Guaçu (SP)'}, 'serie': {'2022': '66970'}}]}]}, {'id': '5936', 'variavel': 'Variação absoluta da população residente 2010 compatibilizada', 'unidade': 'Pessoas', 'resultados': [{'classificacoes': [], 'series': [{'localidade': {'id': '3515103', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Embu-Guaçu (SP)'}, 'serie': {'2022': '4201'}}]}]}, {'id': '10605', 'variavel': 'Taxa de crescimento geométrico', 'unidade': '%', 'resultados': [{'classificacoes': [], 'series': [{'localidade': {'id': '3515103', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Embu-Guaçu (SP)'}, 'serie': {'2022': '0.54'}}]}]}]


In [26]:
municipio = 'Embu-Guaçu'
uf = 'SP'

dict_result = {'municipio':municipio,'uf':uf}

for result in data_json:
    print(result)
    

    var_name = result['variavel']
    var_value = list(result['resultados'][0]['series'][0]['serie'].values())[0]

    dict_result[var_name] = var_value

df_censo = pd.DataFrame([dict_result])
display(df_censo)

df_censo.to_csv('./data/processed/populacao_embu_guacu_2022_crescimento.csv', index=False)

{'id': '93', 'variavel': 'População residente', 'unidade': 'Pessoas', 'resultados': [{'classificacoes': [], 'series': [{'localidade': {'id': '3515103', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Embu-Guaçu (SP)'}, 'serie': {'2022': '66970'}}]}]}
{'id': '5936', 'variavel': 'Variação absoluta da população residente 2010 compatibilizada', 'unidade': 'Pessoas', 'resultados': [{'classificacoes': [], 'series': [{'localidade': {'id': '3515103', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Embu-Guaçu (SP)'}, 'serie': {'2022': '4201'}}]}]}
{'id': '10605', 'variavel': 'Taxa de crescimento geométrico', 'unidade': '%', 'resultados': [{'classificacoes': [], 'series': [{'localidade': {'id': '3515103', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Embu-Guaçu (SP)'}, 'serie': {'2022': '0.54'}}]}]}


,municipio,uf,População residente,Variação absoluta da população residente 2010 compatibilizada,Taxa de crescimento geométrico
0,Embu-Guaçu,SP,66970,4201,0.54


In [27]:
dict_result

{'municipio': 'Embu-Guaçu',
 'uf': 'SP',
 'População residente': '66970',
 'Variação absoluta da população residente 2010 compatibilizada': '4201',
 'Taxa de crescimento geométrico': '0.54'}

In [28]:
for result in data_json:
    print(result)

{'id': '93', 'variavel': 'População residente', 'unidade': 'Pessoas', 'resultados': [{'classificacoes': [], 'series': [{'localidade': {'id': '3515103', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Embu-Guaçu (SP)'}, 'serie': {'2022': '66970'}}]}]}
{'id': '5936', 'variavel': 'Variação absoluta da população residente 2010 compatibilizada', 'unidade': 'Pessoas', 'resultados': [{'classificacoes': [], 'series': [{'localidade': {'id': '3515103', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Embu-Guaçu (SP)'}, 'serie': {'2022': '4201'}}]}]}
{'id': '10605', 'variavel': 'Taxa de crescimento geométrico', 'unidade': '%', 'resultados': [{'classificacoes': [], 'series': [{'localidade': {'id': '3515103', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Embu-Guaçu (SP)'}, 'serie': {'2022': '0.54'}}]}]}


# - 9606- População residente, por cor ou raça, segundo o sexo e a idade

In [29]:
url_final = 'https://servicodados.ibge.gov.br/api/v3/agregados/9606/periodos/2022/variaveis/93?localidades=N6[3515103]&classificacao=86[2776,2777,2778,2779,2780]|2[4,5]|287[93070,93084,93085,93086,93087,93088,93089,93090,93091,93092,93093,93094,93095,93096,93097,93098,49108,49109,60040,60041,6653]'

In [30]:
response = requests.get(url_final)
data_json = response.json()

print(data_json)

[{'id': '93', 'variavel': 'População residente', 'unidade': 'Pessoas', 'resultados': [{'classificacoes': [{'id': '86', 'nome': 'Cor ou raça', 'categoria': {'2776': 'Branca'}}, {'id': '2', 'nome': 'Sexo', 'categoria': {'4': 'Homens'}}, {'id': '287', 'nome': 'Idade', 'categoria': {'93070': '0 a 4 anos'}}], 'series': [{'localidade': {'id': '3515103', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Embu-Guaçu (SP)'}, 'serie': {'2022': '1138'}}]}, {'classificacoes': [{'id': '86', 'nome': 'Cor ou raça', 'categoria': {'2776': 'Branca'}}, {'id': '2', 'nome': 'Sexo', 'categoria': {'4': 'Homens'}}, {'id': '287', 'nome': 'Idade', 'categoria': {'93084': '5 a 9 anos'}}], 'series': [{'localidade': {'id': '3515103', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Embu-Guaçu (SP)'}, 'serie': {'2022': '1232'}}]}, {'classificacoes': [{'id': '86', 'nome': 'Cor ou raça', 'categoria': {'2776': 'Branca'}}, {'id': '2', 'nome': 'Sexo', 'categoria': {'4': 'Homens'}}, {'id': '287', 'nome': 'Idade', 'c

In [31]:
municipio = 'Embu-Guaçu'
uf = 'SP'

list_results = []

for result in data_json[0]['resultados']:
    dict_result = {'municipio':municipio,'uf':uf}

    for classif in result['classificacoes']:
        var_name = classif['nome']
        var_class = list(classif['categoria'].values())[0]

        try:
            var_value = float(list(result['series'][-1]['serie'].values())[0])
        except:
            var_value = 0
        
        dict_result[var_name] = var_class
        print(var_name, var_class, var_value)

    dict_result['habitantes'] = var_value
    list_results.append(dict_result)
    
df_censo = pd.DataFrame(list_results)
#df_censo.columns = ['municipio','uf','sexo','idade','declaracao_idade','habitantes']
#df_censo.drop(columns=['declaracao_idade'], inplace=True)
#df_censo = df_censo.query('idade != "Total" & sexo != "Total"')

df_censo.to_csv('./data/processed/populacao_embu_guacu_2022_raca_sexo_idade.csv', index=False)

Cor ou raça Branca 1138.0
Sexo Homens 1138.0
Idade 0 a 4 anos 1138.0
Cor ou raça Branca 1232.0
Sexo Homens 1232.0
Idade 5 a 9 anos 1232.0
Cor ou raça Branca 1150.0
Sexo Homens 1150.0
Idade 10 a 14 anos 1150.0
Cor ou raça Branca 1121.0
Sexo Homens 1121.0
Idade 15 a 19 anos 1121.0
Cor ou raça Branca 1112.0
Sexo Homens 1112.0
Idade 20 a 24 anos 1112.0
Cor ou raça Branca 1039.0
Sexo Homens 1039.0
Idade 25 a 29 anos 1039.0
Cor ou raça Branca 1016.0
Sexo Homens 1016.0
Idade 30 a 34 anos 1016.0
Cor ou raça Branca 1018.0
Sexo Homens 1018.0
Idade 35 a 39 anos 1018.0
Cor ou raça Branca 1158.0
Sexo Homens 1158.0
Idade 40 a 44 anos 1158.0
Cor ou raça Branca 999.0
Sexo Homens 999.0
Idade 45 a 49 anos 999.0
Cor ou raça Branca 940.0
Sexo Homens 940.0
Idade 50 a 54 anos 940.0
Cor ou raça Branca 760.0
Sexo Homens 760.0
Idade 55 a 59 anos 760.0
Cor ou raça Branca 771.0
Sexo Homens 771.0
Idade 60 a 64 anos 771.0
Cor ou raça Branca 656.0
Sexo Homens 656.0
Idade 65 a 69 anos 656.0
Cor ou raça Branca 414.0


In [32]:
df_censo

,municipio,uf,Cor ou raça,Sexo,Idade,habitantes
0,Embu-Guaçu,SP,Branca,Homens,0 a 4 anos,1138.0
1,Embu-Guaçu,SP,Branca,Homens,5 a 9 anos,1232.0
2,Embu-Guaçu,SP,Branca,Homens,10 a 14 anos,1150.0
3,Embu-Guaçu,SP,Branca,Homens,15 a 19 anos,1121.0
4,Embu-Guaçu,SP,Branca,Homens,20 a 24 anos,1112.0
...,...,...,...,...,...,...
205,Embu-Guaçu,SP,Indígena,Mulheres,80 a 84 anos,1.0
206,Embu-Guaçu,SP,Indígena,Mulheres,85 a 89 anos,1.0
207,Embu-Guaçu,SP,Indígena,Mulheres,90 a 94 anos,1.0
208,Embu-Guaçu,SP,Indígena,Mulheres,95 a 99 anos,0.0


# 9397- Moradores em domicílios particulares permanentes ocupados, por existência de banheiro ou sanitário e número de banheiros de uso exclusivo do domicílio e por tipo de esgotamento sanitário, segundo grupos de idade e cor ou raça

In [52]:
url_final = 'https://servicodados.ibge.gov.br/api/v3/agregados/9397/periodos/-6/variaveis/382?localidades=N6[3515103]&classificacao=458[12032,72118,72119,12046]|11558[72110,72111,72112,72113,92858,72114,72115,92861]|86[2776,2777,2778,2779,2780]'

In [53]:
response = requests.get(url_final)
data_json = response.json()

print(data_json)

[{'id': '382', 'variavel': 'Moradores em domicílios particulares permanentes ocupados', 'unidade': 'Pessoas', 'resultados': [{'classificacoes': [{'id': '458', 'nome': 'Existência de banheiro ou sanitário e número de banheiros de uso exclusivo do domicílio', 'categoria': {'12032': 'Tinham banheiro de uso exclusivo do domicílio'}}, {'id': '11558', 'nome': 'Tipo de esgotamento sanitário', 'categoria': {'72110': 'Rede geral ou pluvial'}}, {'id': '86', 'nome': 'Cor ou raça', 'categoria': {'2776': 'Branca'}}, {'id': '58', 'nome': 'Grupo de idade', 'categoria': {'95253': 'Total'}}], 'series': [{'localidade': {'id': '3515103', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Embu-Guaçu (SP)'}, 'serie': {'2022': '14306'}}]}, {'classificacoes': [{'id': '458', 'nome': 'Existência de banheiro ou sanitário e número de banheiros de uso exclusivo do domicílio', 'categoria': {'12032': 'Tinham banheiro de uso exclusivo do domicílio'}}, {'id': '11558', 'nome': 'Tipo de esgotamento sanitário', 'categ

In [54]:
municipio = 'Embu-Guaçu'
uf = 'SP'

list_results = []

for result in data_json[0]['resultados']:
    dict_result = {'municipio':municipio,'uf':uf}

    for classif in result['classificacoes']:
        var_name = classif['nome']
        var_class = list(classif['categoria'].values())[0]

        try:
            var_value = float(list(result['series'][-1]['serie'].values())[0])
        except:
            var_value = 0
        
        dict_result[var_name] = var_class
        print(var_name, var_class, var_value)

    dict_result['habitantes'] = var_value
    list_results.append(dict_result)
    
df_censo = pd.DataFrame(list_results)
#df_censo.columns = ['municipio','uf','sexo','idade','declaracao_idade','habitantes']
#df_censo.drop(columns=['declaracao_idade'], inplace=True)
#df_censo = df_censo.query('idade != "Total" & sexo != "Total"')

df_censo.to_csv('./data/processed/ibge/populacao_embu_guacu_2022_banheiro_saneaento_raca.csv', index=False)

Existência de banheiro ou sanitário e número de banheiros de uso exclusivo do domicílio Tinham banheiro de uso exclusivo do domicílio 14306.0
Tipo de esgotamento sanitário Rede geral ou pluvial 14306.0
Cor ou raça Branca 14306.0
Grupo de idade Total 14306.0
Existência de banheiro ou sanitário e número de banheiros de uso exclusivo do domicílio Tinham banheiro de uso exclusivo do domicílio 2401.0
Tipo de esgotamento sanitário Rede geral ou pluvial 2401.0
Cor ou raça Preta 2401.0
Grupo de idade Total 2401.0
Existência de banheiro ou sanitário e número de banheiros de uso exclusivo do domicílio Tinham banheiro de uso exclusivo do domicílio 232.0
Tipo de esgotamento sanitário Rede geral ou pluvial 232.0
Cor ou raça Amarela 232.0
Grupo de idade Total 232.0
Existência de banheiro ou sanitário e número de banheiros de uso exclusivo do domicílio Tinham banheiro de uso exclusivo do domicílio 11488.0
Tipo de esgotamento sanitário Rede geral ou pluvial 11488.0
Cor ou raça Parda 11488.0
Grupo de i

In [55]:
df_censo.sort_values(by='habitantes')

,municipio,uf,Existência de banheiro ou sanitário e número de banheiros de uso exclusivo do domicílio,Tipo de esgotamento sanitário,Cor ou raça,Grupo de idade,habitantes
79,Embu-Guaçu,SP,Apenas banheiro de uso comum a mais de um domi...,Não tinham banheiro nem sanitário,Indígena,Total,0.0
101,Embu-Guaçu,SP,"Apenas sanitário ou buraco para dejeções, incl...",Vala,Preta,Total,0.0
102,Embu-Guaçu,SP,"Apenas sanitário ou buraco para dejeções, incl...",Vala,Amarela,Total,0.0
103,Embu-Guaçu,SP,"Apenas sanitário ou buraco para dejeções, incl...",Vala,Parda,Total,0.0
104,Embu-Guaçu,SP,"Apenas sanitário ou buraco para dejeções, incl...",Vala,Indígena,Total,0.0
...,...,...,...,...,...,...,...
13,Embu-Guaçu,SP,Tinham banheiro de uso exclusivo do domicílio,Fossa séptica ou fossa filtro não ligada à rede,Parda,Total,7695.0
18,Embu-Guaçu,SP,Tinham banheiro de uso exclusivo do domicílio,Fossa rudimentar ou buraco,Parda,Total,7729.0
10,Embu-Guaçu,SP,Tinham banheiro de uso exclusivo do domicílio,Fossa séptica ou fossa filtro não ligada à rede,Branca,Total,8084.0
3,Embu-Guaçu,SP,Tinham banheiro de uso exclusivo do domicílio,Rede geral ou pluvial,Parda,Total,11488.0


In [56]:
df_censo['habitantes'].sum()

66631.0

# Dados educação - API simcaq (UFPR)

## Alunos matriculados

In [109]:
url = 'https://simcaq.c3sl.ufpr.br/api/v1/enrollment?dims=education_level,adm_dependency&filter=city:3515103,min_year:2020'

response = requests.get(url)
data_json = response.json()

In [110]:
municipio = 'Embu-Guaçu'
uf = 'SP'

list_results = []

for result in data_json['result']:
    dict_result = result.copy()
    dict_result['municipio'] = municipio
    dict_result['uf'] = uf

    list_results.append(dict_result)
    
df_censo = pd.DataFrame(list_results)

display(df_censo.head())

,total,year,adm_dependency_id,education_level_id,adm_dependency_name,education_level_name,municipio,uf
0,399,2020,2,14,Estadual,Ensino Fundamental de 9 anos - 1º Ano,Embu-Guaçu,SP
1,559,2020,2,15,Estadual,Ensino Fundamental de 9 anos - 2º Ano,Embu-Guaçu,SP
2,574,2020,2,16,Estadual,Ensino Fundamental de 9 anos - 3º Ano,Embu-Guaçu,SP
3,511,2020,2,17,Estadual,Ensino Fundamental de 9 anos - 4º Ano,Embu-Guaçu,SP
4,492,2020,2,18,Estadual,Ensino Fundamental de 9 anos - 5º Ano,Embu-Guaçu,SP


In [111]:
# Ajuste nas categorias
print(sorted(df_censo['education_level_name'].unique()))
print(sorted(df_censo['adm_dependency_name'].unique()))

['Curso Técnico Integrado (Ensino Médio Integrado) 1ª Série', 'EJA - Ensino Fundamental - Anos Finais', 'EJA - Ensino Fundamental - Anos Iniciais EJA - Ensino Fundamental - Anos Iniciais', 'EJA - Ensino Médio', 'Educação Infantil - Creche', 'Educação Infantil - Pré-Escola', 'Ensino Fundamental de 9 anos - 1º Ano', 'Ensino Fundamental de 9 anos - 2º Ano', 'Ensino Fundamental de 9 anos - 3º Ano', 'Ensino Fundamental de 9 anos - 4º Ano', 'Ensino Fundamental de 9 anos - 5º Ano', 'Ensino Fundamental de 9 anos - 6º Ano', 'Ensino Fundamental de 9 anos - 7º Ano', 'Ensino Fundamental de 9 anos - 8º Ano', 'Ensino Fundamental de 9 anos - 9º Ano', 'Ensino Médio - 1ª Série', 'Ensino Médio - 2ª Série', 'Ensino Médio - 3ª Série']
['Estadual', 'Municipal', 'Privada']


In [112]:
dict_translate_anos = {'Curso Técnico Integrado (Ensino Médio Integrado) 1ª Série':'Ensino Médio',
                       'EJA - Ensino Fundamental - Anos Finais':'EJA',
                        'EJA - Ensino Fundamental - Anos Iniciais EJA - Ensino Fundamental - Anos Iniciais':'EJA',
                        'EJA - Ensino Médio':'EJA',
                        'Educação Infantil - Creche':'Educação Infantil',
                        'Educação Infantil - Pré-Escola':'Educação Infantil',
                        'Ensino Fundamental de 9 anos - 1º Ano':'Ensino Fundamental (anos iniciais)',
                        'Ensino Fundamental de 9 anos - 2º Ano':'Ensino Fundamental (anos iniciais)',
                        'Ensino Fundamental de 9 anos - 3º Ano':'Ensino Fundamental (anos iniciais)',
                        'Ensino Fundamental de 9 anos - 4º Ano':'Ensino Fundamental (anos iniciais)',
                        'Ensino Fundamental de 9 anos - 5º Ano':'Ensino Fundamental (anos iniciais)',
                        'Ensino Fundamental de 9 anos - 6º Ano':'Ensino Fundamental (anos finais)',
                        'Ensino Fundamental de 9 anos - 7º Ano':'Ensino Fundamental (anos finais)',
                        'Ensino Fundamental de 9 anos - 8º Ano':'Ensino Fundamental (anos finais)',
                        'Ensino Fundamental de 9 anos - 9º Ano':'Ensino Fundamental (anos finais)',
                        'Ensino Médio - 1ª Série':'Ensino Médio',
                        'Ensino Médio - 2ª Série':'Ensino Médio',
                        'Ensino Médio - 3ª Série':'Ensino Médio'}

df_censo['education_level_cat'] = df_censo['education_level_name'].map(dict_translate_anos)
df_censo['education_level_cat'].value_counts()

education_level_cat
Ensino Fundamental (anos iniciais)    15
Ensino Fundamental (anos finais)      12
Ensino Médio                           7
Educação Infantil                      4
EJA                                    3
Name: count, dtype: int64

In [114]:
df_censo.to_csv('./data/processed/mec_inep/alunos_matriculados_embu_guacu_2020.csv', index=False)

## Número de escolas

In [115]:
url = 'https://simcaq.c3sl.ufpr.br/api/v1/school/count?dims=adm_dependency&filter=city:3515103,year:2020'

response = requests.get(url)
data_json = response.json()

print(data_json)

{'result': [{'total': 20, 'name': 'Brasil', 'year': 2020, 'adm_dependency_id': 2, 'adm_dependency_name': 'Estadual'}, {'total': 33, 'name': 'Brasil', 'year': 2020, 'adm_dependency_id': 3, 'adm_dependency_name': 'Municipal'}, {'total': 10, 'name': 'Brasil', 'year': 2020, 'adm_dependency_id': 4, 'adm_dependency_name': 'Privada'}]}


In [118]:
municipio = 'Embu-Guaçu'
uf = 'SP'

list_results = []

for result in data_json['result']:
    dict_result = result.copy()
    dict_result['municipio'] = municipio
    dict_result['uf'] = uf

    list_results.append(dict_result)
    
df_censo = pd.DataFrame(list_results)

df_censo.to_csv('./data/processed/mec_inep/escolas_embu_guacu_2020.csv', index=False)

## Número de professores

In [136]:
url = 'https://simcaq.c3sl.ufpr.br/api/v1/teacher?dims=adm_dependency&filter=city:3515103,min_year:2020'

response = requests.get(url)
data_json = response.json()

print(data_json)

{'result': [{'total': 423, 'name': 'Brasil', 'year': 2020, 'adm_dependency_id': 2, 'adm_dependency_name': 'Estadual'}, {'total': 296, 'name': 'Brasil', 'year': 2020, 'adm_dependency_id': 3, 'adm_dependency_name': 'Municipal'}, {'total': 160, 'name': 'Brasil', 'year': 2020, 'adm_dependency_id': 4, 'adm_dependency_name': 'Privada'}]}


In [137]:
municipio = 'Embu-Guaçu'
uf = 'SP'

list_results = []

for result in data_json['result']:
    print(result)
    dict_result = result.copy()
    dict_result['municipio'] = municipio
    dict_result['uf'] = uf

    list_results.append(dict_result)
    
df_censo = pd.DataFrame(list_results)

df_censo.to_csv('./data/processed/mec_inep/professores_embu_guacu_2020.csv', index=False)

{'total': 423, 'name': 'Brasil', 'year': 2020, 'adm_dependency_id': 2, 'adm_dependency_name': 'Estadual'}
{'total': 296, 'name': 'Brasil', 'year': 2020, 'adm_dependency_id': 3, 'adm_dependency_name': 'Municipal'}
{'total': 160, 'name': 'Brasil', 'year': 2020, 'adm_dependency_id': 4, 'adm_dependency_name': 'Privada'}


In [138]:
df_censo

,total,name,year,adm_dependency_id,adm_dependency_name,municipio,uf
0,423,Brasil,2020,2,Estadual,Embu-Guaçu,SP
1,296,Brasil,2020,3,Municipal,Embu-Guaçu,SP
2,160,Brasil,2020,4,Privada,Embu-Guaçu,SP
